In [ ]:
import os
import torch

import utils

%load_ext autoreload
%autoreload 2

CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(os.listdir("../data/"))


In [ ]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset = GeorgianLanguageDatasetLoader("../data/kawiki.txt", 5, device)

In [ ]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

In [ ]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

In [ ]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

In [ ]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

In [ ]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [ ]:
train_data_batched.shape, train_data_batched.device

In [ ]:
x, y = utils.get_batch(train_data_batched, 1)

In [ ]:
x.shape, y.shape, train_data_batched.shape

In [ ]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

In [ ]:
x[0], y[0]

In [ ]:
%reload_ext autoreload

model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

In [ ]:
text = utils.generate_text(model,device, dataset.vocab, 'იკვლევდა ევროპის და განსაკუთრებით', 20)

In [ ]:
print(text)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
geo_model = utils.GeorgianWord2Vec()
geo_model.train("../data/kawiki.txt")

In [ ]:
embeddings = torch.FloatTensor(geo_model.get_model().wv.vectors)
embeddings.shape

In [ ]:
model = utils.LSTMModel(128, 600, len(vocab), device, num_layers=1, embeddings=embeddings).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

In [ ]:
text = utils.generate_text(model,device, dataset.vocab, 'პოლიტიკური კარიერა', 10)
print(text)